In [1]:
import langchain
import json
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools.pubmed.tool import PubmedQueryRun
from Bio import Entrez

Test LLM

In [2]:
llm = Ollama(model='llama3',temperature=0.1,base_url='http://192.168.0.64:11434')

Generar search queries

In [3]:
symptoms = "Ayer entrené en el gimnasio y tengo un hormigueo que me empieza en la parte baja de la espalda y baja por la pierna derecha."

pm_search_prompt = 'Eres un investigador avanzado en fisioterapia. Tu tarea es buscar papers en PubMed para encontrar un \
                    diagnóstico y tratamiento de fisioterapia para los siguientes síntomas: "{symptoms}" \
                    Dime 10 entradas de búsqueda que pondrías en la web de pubmed para encontrar papers sobre los síntomas. \
                    No emplees combinaciones como OR o AND, dime únicamente las palabras clave de búsqueda.\
                    Devuelve únicamente las entradas de búsqueda como una lista de python, no digas nada más'

pm_search_template = ChatPromptTemplate.from_template(pm_search_prompt)

Function to search pubmed articles

In [4]:
def generate_search(pm_search_template,symptoms,llm):
    pm_search_llm = pm_search_template | llm
    search_entries = pm_search_llm.invoke({'symptoms':symptoms})
    return search_entries
search_entries = generate_search(pm_search_template,symptoms,llm).replace('[','').replace(']','').replace("'","").split(', ')
print(search_entries)

['lower back pain', 'post-exercise soreness', 'delayed onset muscle soreness', 'DOMS', 'lumbar region', 'gluteal region', 'right lower extremity', 'exercise-induced muscle soreness', 'muscle strain', 'soft tissue injury']


Search in pubmed API

In [5]:
def search(entry):
    Entrez.email = 'oscarvallslozano@gmail.com'
    handle = Entrez.esearch(db='pubmed',
                            sort='relevance',
                            retmax='5',
                            retmode='xml',
                            term=entry)
    results = Entrez.read(handle)
    return results

def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'oscarvallslozano@gmail.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results


results = search(search_entries[0])
id_list = results['IdList']
papers = fetch_details(id_list)
for i, paper in enumerate(papers['PubmedArticle']):
        print("{}) {}".format(i+1, paper['MedlineCitation']['Article']['ArticleTitle']))


1) Neck Pain and Lower Back Pain.
2) Chronic Lower Back Pain in Weight Lifters: Epidemiology, Evaluation, and Management.
3) Clinical outcomes and cost-effectiveness of massage chair therapy versus basic physiotherapy in lower back pain patients: A randomized controlled trial.
4) Acute Lumbar Back Pain.
5) Treatment and ergonomics training of work-related lower back pain and body posture problems for nurses.


In [38]:
for paper in papers['PubmedArticle']:
    print(paper['MedlineCitation']['Article']['Abstract']['AbstractText'])

['Neck pain is the fourth leading cause of disability. Acute neck pain largely resolves within 2\xa0months. History and physical examination play a key role in ruling out some of the more serious causes for neck pain. The evidence for pharmacologic interventions for acute and chronic musculoskeletal neck pain is limited. Lower back pain is the leading cause of disability and productivity loss. Consultation with a physical medicine and rehabilitation spine specialist within 48\xa0hours for acute pain and within 10\xa0days for all patients with lower back pain may significantly decrease rate of surgical interventions and increase patient satisfaction.']
["» Chronic lower back pain (LBP) is common in both nonathletes and weight lifters, but the diagnosis and treatment should be approached differently in these 2 populations based on the unique movement patterns causing the pain.» Injury rates of weight lifters are far less than those of contact sports, ranging from 1.0 to 4.4 injuries per 